# Set up

In [1]:
# update path with data dir
import sys
sys.path.append('../../data/')

In [2]:
import warnings
# warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import patsy
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import PredefinedSplit, GridSearchCV, cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.backend import clear_session
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

/home/tom/footbot/venv/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Getting data

In [4]:
file_name = '../csvs/element_gameweek_features_v05.csv'

In [5]:
element_gameweek_df = pd.read_csv(file_name)

In [6]:
current_event = 20

In [7]:
element_gameweek_df =\
element_gameweek_df[
    (element_gameweek_df['event'] <= current_event)
]

In [8]:
element_gameweek_df.fillna(3, inplace=True)

In [9]:
element_gameweek_df = element_gameweek_df[element_gameweek_df['rolling_avg_minutes_element_p3'] >= 60]

# Modelling data

## Splitting data

In [10]:
last_train_event = 15

In [11]:
test_fold = [-1 if i <= last_train_event else 0 for i in element_gameweek_df['event']]
ps = PredefinedSplit(test_fold)

In [12]:
for train_index, test_index in ps.split():
    element_gameweek_df_train, element_gameweek_df_test = \
    element_gameweek_df.copy().iloc[train_index], element_gameweek_df.copy().iloc[test_index]

## Building model

In [13]:
formula = \
'total_points ~ C(element) + C(element_type) + C(team) + C(opposition_team) + ' + \
' + '.join([i for i in element_gameweek_df.columns if i not in
 [
     'safe_web_name',
     'element',
     'event',
     'fixture',
     'team',
     'opposition_team',
     'element_type',
     'minutes',
     'rolling_value_points',
     'total_points',
     'rolling_avg_total_points_against_opposition_team_element_type_home',
     'rolling_avg_total_points_against_opposition_team_element_type_away'
 ]])

In [14]:
print(formula)

total_points ~ C(element) + C(element_type) + C(team) + C(opposition_team) + was_home + is_sunday + is_weekday + is_early + is_late + value + rolling_avg_total_points_element + rolling_max_total_points_element + has_high_points_ever_element + rolling_avg_bps_element + rolling_avg_bonus_element + rolling_avg_goals_scored_element + rolling_avg_minutes_element + rolling_avg_clean_sheets_element + rolling_avg_completed_passes_element + rolling_avg_recoveries_element + rolling_avg_assists_element + rolling_avg_key_passes_element + rolling_avg_winning_goals_element + rolling_avg_tackled_element + rolling_avg_dribbles_element + rolling_avg_clearances_blocks_interceptions_element + rolling_avg_big_chances_created_element + rolling_avg_target_missed_element + rolling_avg_fouls_element + rolling_avg_tackles_element + rolling_avg_open_play_crosses_element + rolling_avg_offside_element + rolling_avg_big_chances_missed_element + rolling_avg_saves_element + rolling_avg_goals_conceded_element + rolli

In [15]:
y, X = patsy.dmatrices(formula, element_gameweek_df, return_type='matrix')

In [16]:
for train_index, test_index in ps.split():
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [17]:
n_features = X.shape[1]

In [18]:
X_sel = X
X_train_sel = X_train
X_test_sel = X_test

## Tuning model

### Helpers

In [19]:
def create_model(hidden_nodes,
                 hidden_layers,
                 dropout_rate,
                 weight_constraint
                ):
    model = Sequential()
    model.add(Dense(hidden_nodes,
                    activation='relu',
                    kernel_constraint=MaxNorm(weight_constraint)
                   ))
    
    
    for i in range(0, hidden_layers):
        model.add(Dropout(dropout_rate))
        model.add(Dense(hidden_nodes, activation='relu', kernel_constraint=MaxNorm(weight_constraint)))
        
        

    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

### GP tuner

In [20]:
space = [
    Real(1, 500, 'uniform', name='hidden_nodes'),
    Integer(0, 5, name='hidden_layers'),
    Real(20, 1000, 'log-uniform', name='epochs'),
    Real(20, 150, 'log-uniform', name='batch_size'),
    Real(0.0, 0.9, 'uniform', name='dropout_rate'),
    Integer(1, 5, name='weight_constraint')
]

In [21]:
@use_named_args(space)
def objective(hidden_nodes,
              hidden_layers,
              epochs,
              batch_size,
              dropout_rate,
              weight_constraint
             ):
    
    model = create_model(
        hidden_nodes=int(hidden_nodes),
        hidden_layers=int(hidden_layers),
        dropout_rate=dropout_rate,
        weight_constraint=weight_constraint
    )
    
    model.fit(X_train_sel,
              y_train.ravel(),
              epochs=int(epochs),
              batch_size=int(batch_size))
    
    y_pred = model.predict(X_test_sel).flatten()
    test_loss = mean_squared_error(y_test, y_pred)

    del model
    clear_session()
    
    return test_loss

In [ ]:
res_gp_arr = []
for i in range(1, 11):
    print('minimisation iteration', i)
    res_gp = gp_minimize(objective, space, n_calls=30, verbose=2)
    res_gp_arr.append(res_gp)

minimisation iteration 1
Iteration No: 1 started. Evaluating function at random point.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Epoch 1/911
2696/2696 [==============================] - 2s 647us/sample - loss: 1470.4773
Epoch 2/911
2696/2696 [==============================] - 1s 230us/sample - loss: 154.1938
Epoch 3/911
2696/2696 [==============================] - 1s 309us/sample - loss: 79.6963
Epoch 4/911
2696/2696 [==============================] - 1s 265us/sample - loss: 52.4694
Epoch 5/911
2696/2696 [==============================] - 1s 207us/sample - loss: 35.1331
Epoch 6/911
2696/2696 [==============================] - 1s 267us/sample - loss: 26.6331
Epoch 7/911
2696/2696 [==============================] - 1s 216us/sample - loss: 21.9747 - lo

2696/2696 [==============================] - 1s 248us/sample - loss: 9.2192
Epoch 79/911
2696/2696 [==============================] - 1s 290us/sample - loss: 9.2562
Epoch 80/911
2696/2696 [==============================] - 1s 274us/sample - loss: 9.2759
Epoch 81/911
2696/2696 [==============================] - 0s 178us/sample - loss: 9.2109
Epoch 82/911
2696/2696 [==============================] - 1s 201us/sample - loss: 9.2678
Epoch 83/911
2696/2696 [==============================] - 1s 260us/sample - loss: 9.2849
Epoch 84/911
2696/2696 [==============================] - 0s 155us/sample - loss: 9.2974
Epoch 85/911
2696/2696 [==============================] - 0s 133us/sample - loss: 9.2461
Epoch 86/911
2696/2696 [==============================] - 1s 245us/sample - loss: 9.2250
Epoch 87/911
2696/2696 [==============================] - 1s 225us/sample - loss: 9.2768
Epoch 88/911
2696/2696 [==============================] - 1s 231us/sample - loss: 9.2482
Epoch 89/911
2696/2696 [==========

2696/2696 [==============================] - 1s 248us/sample - loss: 9.2715
Epoch 170/911
2696/2696 [==============================] - 1s 215us/sample - loss: 9.2778
Epoch 171/911
2696/2696 [==============================] - 0s 129us/sample - loss: 9.2838
Epoch 172/911
2696/2696 [==============================] - 1s 234us/sample - loss: 9.2735
Epoch 173/911
2696/2696 [==============================] - 1s 243us/sample - loss: 9.3241
Epoch 174/911
2696/2696 [==============================] - 1s 248us/sample - loss: 9.2732
Epoch 175/911
2696/2696 [==============================] - 1s 220us/sample - loss: 9.2785
Epoch 176/911
2696/2696 [==============================] - 1s 246us/sample - loss: 9.2771
Epoch 177/911
2696/2696 [==============================] - 0s 152us/sample - loss: 9.2839
Epoch 178/911
2696/2696 [==============================] - 1s 212us/sample - loss: 9.2791
Epoch 179/911
2696/2696 [==============================] - 1s 238us/sample - loss: 9.2759
Epoch 180/911
2696/2696 

2696/2696 [==============================] - 0s 182us/sample - loss: 9.2775
Epoch 261/911
2696/2696 [==============================] - 1s 214us/sample - loss: 9.2870
Epoch 262/911
2696/2696 [==============================] - 0s 152us/sample - loss: 9.2777
Epoch 263/911
2696/2696 [==============================] - 0s 137us/sample - loss: 9.2835
Epoch 264/911
2696/2696 [==============================] - 0s 140us/sample - loss: 9.2862
Epoch 265/911
2696/2696 [==============================] - 0s 116us/sample - loss: 9.2892
Epoch 266/911
2696/2696 [==============================] - 1s 269us/sample - loss: 9.2849
Epoch 267/911
2696/2696 [==============================] - 1s 245us/sample - loss: 9.2814
Epoch 268/911
2696/2696 [==============================] - 0s 99us/sample - loss: 9.2798
Epoch 269/911
2696/2696 [==============================] - 0s 133us/sample - loss: 9.2780
Epoch 270/911
2696/2696 [==============================] - 0s 135us/sample - loss: 9.2769
Epoch 271/911
2696/2696 [

2696/2696 [==============================] - 1s 215us/sample - loss: 9.2863
Epoch 352/911
2696/2696 [==============================] - 1s 206us/sample - loss: 9.2951
Epoch 353/911
2696/2696 [==============================] - 1s 224us/sample - loss: 9.2838
Epoch 354/911
2696/2696 [==============================] - 1s 216us/sample - loss: 9.2818
Epoch 355/911
2696/2696 [==============================] - 1s 227us/sample - loss: 9.2847
Epoch 356/911
2696/2696 [==============================] - 0s 184us/sample - loss: 9.2921
Epoch 357/911
2696/2696 [==============================] - 0s 180us/sample - loss: 9.2834
Epoch 358/911
2696/2696 [==============================] - 1s 281us/sample - loss: 9.2780
Epoch 359/911
2696/2696 [==============================] - 1s 221us/sample - loss: 9.2895
Epoch 360/911
2696/2696 [==============================] - 1s 222us/sample - loss: 9.2836
Epoch 361/911
2696/2696 [==============================] - 1s 211us/sample - loss: 9.2828
Epoch 362/911
2696/2696 

...

2696/2696 [==============================] - 0s 38us/sample - loss: 1.8120
Epoch 271/530
2696/2696 [==============================] - 0s 35us/sample - loss: 1.7140
Epoch 272/530
2696/2696 [==============================] - 0s 32us/sample - loss: 1.8890
Epoch 273/530
2696/2696 [==============================] - 0s 24us/sample - loss: 1.9267
Epoch 274/530
2696/2696 [==============================] - 0s 38us/sample - loss: 1.7730
Epoch 275/530
2696/2696 [==============================] - 0s 26us/sample - loss: 1.9225
Epoch 276/530
2696/2696 [==============================] - 0s 25us/sample - loss: 2.2338
Epoch 277/530
2696/2696 [==============================] - 0s 28us/sample - loss: 1.6630
Epoch 278/530
2696/2696 [==============================] - 0s 37us/sample - loss: 1.7141
Epoch 279/530
2696/2696 [==============================] - 0s 29us/sample - loss: 1.7746
Epoch 280/530
2696/2696 [==============================] - 0s 45us/sample - loss: 2.2772
Epoch 281/530
2696/2696 [==========

2696/2696 [==============================] - 0s 30us/sample - loss: 1.2053
Epoch 363/530
2696/2696 [==============================] - 0s 33us/sample - loss: 0.9692
Epoch 364/530
2696/2696 [==============================] - 0s 39us/sample - loss: 0.9748
Epoch 365/530
2696/2696 [==============================] - 0s 35us/sample - loss: 0.9385
Epoch 366/530
2696/2696 [==============================] - 0s 49us/sample - loss: 1.0602
Epoch 367/530
2696/2696 [==============================] - 0s 56us/sample - loss: 1.0701
Epoch 368/530
2696/2696 [==============================] - 0s 37us/sample - loss: 0.9973
Epoch 369/530
2696/2696 [==============================] - 0s 39us/sample - loss: 0.8792
Epoch 370/530
2696/2696 [==============================] - 0s 28us/sample - loss: 0.8472
Epoch 371/530
2696/2696 [==============================] - 0s 28us/sample - loss: 0.8936
Epoch 372/530
2696/2696 [==============================] - 0s 44us/sample - loss: 0.9622
Epoch 373/530
2696/2696 [==========

2696/2696 [==============================] - 0s 36us/sample - loss: 0.9695
Epoch 455/530
2696/2696 [==============================] - 0s 41us/sample - loss: 0.6072
Epoch 456/530
2696/2696 [==============================] - 0s 49us/sample - loss: 0.5045
Epoch 457/530
2696/2696 [==============================] - 0s 45us/sample - loss: 0.4768
Epoch 458/530
2696/2696 [==============================] - 0s 53us/sample - loss: 0.4360
Epoch 459/530
2696/2696 [==============================] - 0s 53us/sample - loss: 0.7502
Epoch 460/530
2696/2696 [==============================] - 0s 27us/sample - loss: 0.8364
Epoch 461/530
2696/2696 [==============================] - 0s 46us/sample - loss: 0.6065
Epoch 462/530
2696/2696 [==============================] - 0s 41us/sample - loss: 0.4940
Epoch 463/530
2696/2696 [==============================] - 0s 29us/sample - loss: 0.4916
Epoch 464/530
2696/2696 [==============================] - 0s 24us/sample - loss: 0.4270
Epoch 465/530
2696/2696 [==========

2696/2696 [==============================] - 0s 111us/sample - loss: 13.2307
Epoch 15/20
2696/2696 [==============================] - 0s 88us/sample - loss: 12.9966
Epoch 16/20
2696/2696 [==============================] - 0s 128us/sample - loss: 12.7725
Epoch 17/20
2696/2696 [==============================] - 0s 118us/sample - loss: 12.5580
Epoch 18/20
2696/2696 [==============================] - 0s 89us/sample - loss: 12.3532
Epoch 19/20
2696/2696 [==============================] - 0s 87us/sample - loss: 12.1582
Epoch 20/20
2696/2696 [==============================] - 0s 73us/sample - loss: 11.9720
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 11.2402
Function value obtained: 12.8591
Current minimum: 9.5599
Iteration No: 23 started. Searching for the next optimal point.
Epoch 1/125
2696/2696 [==============================] - 1s 433us/sample - loss: 10.2512
Epoch 2/125
2696/2696 [==============================] - 1s 351us/sample - loss: 9.0850
Epoch 3

Epoch 84/125
2696/2696 [==============================] - 1s 354us/sample - loss: 3.9215
Epoch 85/125
2696/2696 [==============================] - 1s 363us/sample - loss: 3.7590
Epoch 86/125
2696/2696 [==============================] - 1s 238us/sample - loss: 3.5845
Epoch 87/125
2696/2696 [==============================] - 1s 316us/sample - loss: 3.5744
Epoch 88/125
2696/2696 [==============================] - 1s 395us/sample - loss: 3.5336
Epoch 89/125
2696/2696 [==============================] - 1s 311us/sample - loss: 3.6072
Epoch 90/125
2696/2696 [==============================] - 1s 379us/sample - loss: 3.4148
Epoch 91/125
2696/2696 [==============================] - 1s 298us/sample - loss: 3.3174
Epoch 92/125
2696/2696 [==============================] - 1s 260us/sample - loss: 3.2363
Epoch 93/125
2696/2696 [==============================] - 1s 365us/sample - loss: 3.3593
Epoch 94/125
2696/2696 [==============================] - 1s 265us/sample - loss: 3.2902
Epoch 95/125
2696/269

In [ ]:
res_gp_df = pd.DataFrame([i.x for i in res_gp_arr],
             columns=[
                 'hidden_nodes', 'hidden_layers', 'epochs', 'batch_size',
                 'dropout_rate', 'weight_constraint'
             ])

res_gp_df['MSE'] = [i.fun for i in res_gp_arr]

res_gp_df.sort_values('MSE')

### Grid search

In [84]:
model = KerasRegressor(build_fn=create_model)

In [114]:
hidden_nodes = [20, 25, 30, 35]
hidden_layers = [1]
epochs = [15, 20, 25]
batch_size = [100, 150]
dropout_rate = [0.01, 0.015, 0.02]
weight_constraint = [3, 4]


parameters = {
    'hidden_nodes': hidden_nodes,
    'hidden_layers': hidden_layers,
    'epochs': epochs,
    'batch_size': batch_size,
    'dropout_rate': dropout_rate,
    'weight_constraint': weight_constraint
}

In [115]:
gs_model = GridSearchCV(
    model,
    parameters,
    scoring=['neg_mean_absolute_error', 'neg_mean_squared_error'],
    refit=False,
    cv=ps,
    error_score=100, n_jobs=1, verbose=2)
gs_model.fit(X_sel, y.ravel())

Fitting 1 folds for each of 144 candidates, totalling 144 fits
[CV] batch_size=100, dropout_rate=0.01, epochs=15, hidden_layers=1, hidden_nodes=20, weight_constraint=3 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/15
2696/2696 [==============================] - 5s 2ms/sample - loss: 25.6371
Epoch 2/15
2696/2696 [==============================] - 0s 24us/sample - loss: 10.2682
Epoch 3/15
2696/2696 [==============================] - 0s 23us/sample - loss: 9.8719
Epoch 4/15
2696/2696 [==============================] - 0s 23us/sample - loss: 9.3597
Epoch 5/15
2696/2696 [==============================] - 0s 24us/sample - loss: 9.3075
Epoch 6/15
2696/2696 [==============================] - 0s 24us/sample - loss: 9.2318
Epoch 7/15
2696/2696 [==============================] - 0s 25us/sample - loss: 8.9056
Epoch 8/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.8603
Epoch 9/15
2696/2696 [==============================] - 0s 26us/sample - loss: 9.0297
Epoch 10/15
2696/2696 [==============================] - 0s 27us/sample - loss: 8.7067
Epoch 11/15
2696/2696 [==============================] - 0s 26us/sample - loss: 8.6547
Epoch 12/15
2696/2696 [============================

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.9s remaining:    0.0s


Epoch 1/15
2696/2696 [==============================] - 5s 2ms/sample - loss: 17.6540
Epoch 2/15
2696/2696 [==============================] - 0s 24us/sample - loss: 9.2164
Epoch 3/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.8120
Epoch 4/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.6965
Epoch 5/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.6403
Epoch 6/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.6896
Epoch 7/15
2696/2696 [==============================] - 0s 24us/sample - loss: 8.5649
Epoch 8/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.4960
Epoch 9/15
2696/2696 [==============================] - 0s 24us/sample - loss: 8.4109
Epoch 10/15
2696/2696 [==============================] - 0s 24us/sample - loss: 8.4096
Epoch 11/15
2696/2696 [==============================] - 0s 23us/sample - loss: 8.3668
Epoch 12/15
2696/2696 [=============================

...

2696/2696 [==============================] - 0s 33us/sample - loss: 8.9130
Epoch 6/20
2696/2696 [==============================] - 0s 33us/sample - loss: 8.8086
Epoch 7/20
2696/2696 [==============================] - 0s 33us/sample - loss: 8.7501
Epoch 8/20
2696/2696 [==============================] - 0s 32us/sample - loss: 8.7917
Epoch 9/20
2696/2696 [==============================] - 0s 34us/sample - loss: 8.6446
Epoch 10/20
2696/2696 [==============================] - 0s 33us/sample - loss: 8.6032
Epoch 11/20
2696/2696 [==============================] - 0s 33us/sample - loss: 8.5394
Epoch 12/20
2696/2696 [==============================] - 0s 33us/sample - loss: 8.4361
Epoch 13/20
2696/2696 [==============================] - 0s 33us/sample - loss: 8.5092
Epoch 14/20
2696/2696 [==============================] - 0s 33us/sample - loss: 8.4172
Epoch 15/20
2696/2696 [==============================] - 0s 33us/sample - loss: 8.3315
Epoch 16/20
2696/2696 [==============================] - 0s

2696/2696 [==============================] - 0s 34us/sample - loss: 8.2051
Epoch 21/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.1738
Epoch 22/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.1031
Epoch 23/25
2696/2696 [==============================] - 0s 34us/sample - loss: 8.1024
Epoch 24/25
2696/2696 [==============================] - 0s 36us/sample - loss: 8.1360
Epoch 25/25
2696/2696 [==============================] - 0s 34us/sample - loss: 8.1342
[CV]  batch_size=150, dropout_rate=0.02, epochs=25, hidden_layers=1, hidden_nodes=25, weight_constraint=4, total=  54.7s
[CV] batch_size=150, dropout_rate=0.02, epochs=25, hidden_layers=1, hidden_nodes=30, weight_constraint=3 
Epoch 1/25
2696/2696 [==============================] - 36s 13ms/sample - loss: 15.7577
Epoch 2/25
2696/2696 [==============================] - 0s 30us/sample - loss: 9.1965
Epoch 3/25
2696/2696 [==============================] - 0s 40us/sample - loss: 9.0992
Epo

2696/2696 [==============================] - 0s 35us/sample - loss: 9.3311
Epoch 5/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.9631
Epoch 6/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.8835
Epoch 7/25
2696/2696 [==============================] - 0s 37us/sample - loss: 8.9222
Epoch 8/25
2696/2696 [==============================] - 0s 37us/sample - loss: 8.7451
Epoch 9/25
2696/2696 [==============================] - 0s 35us/sample - loss: 8.6001
Epoch 10/25
2696/2696 [==============================] - 0s 37us/sample - loss: 8.5924
Epoch 11/25
2696/2696 [==============================] - 0s 43us/sample - loss: 8.6047
Epoch 12/25
2696/2696 [==============================] - 0s 45us/sample - loss: 8.3171
Epoch 13/25
2696/2696 [==============================] - 0s 48us/sample - loss: 8.4383
Epoch 14/25
2696/2696 [==============================] - 0s 39us/sample - loss: 8.2950
Epoch 15/25
2696/2696 [==============================] - 0s 

[Parallel(n_jobs=1)]: Done 144 out of 144 | elapsed: 72.4min finished


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             error_score=100,
             estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f712028d550>,
             iid='warn', n_jobs=1,
             param_grid={'batch_size': [100, 150],
                         'dropout_rate': [0.01, 0.015, 0.02],
                         'epochs': [15, 20, 25], 'hidden_layers': [1],
                         'hidden_nodes': [20, 25, 30, 35],
                         'weight_constraint': [3, 4]},
             pre_dispatch='2*n_jobs', refit=False, return_train_score=False,
             scoring=['neg_mean_absolute_error', 'neg_mean_squared_error'],
             verbose=2)

In [116]:
gs_model_df = pd.DataFrame(gs_model.cv_results_)[[
    'param_hidden_nodes', 'param_hidden_layers', 'param_epochs',
    'param_batch_size', 'param_dropout_rate', 'param_weight_constraint',
    'split0_test_neg_mean_absolute_error',
    'split0_test_neg_mean_squared_error'
]]

gs_model_df

,param_hidden_nodes,param_hidden_layers,param_epochs,param_batch_size,param_dropout_rate,param_weight_constraint,split0_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error
0,20,1,15,100,0.01,3,-2.124033,-9.619317
1,20,1,15,100,0.01,4,-2.110486,-9.736834
2,25,1,15,100,0.01,3,-2.088165,-9.728637
3,25,1,15,100,0.01,4,-2.186907,-9.500431
4,30,1,15,100,0.01,3,-2.242478,-9.546927
5,30,1,15,100,0.01,4,-2.266384,-9.568992
6,35,1,15,100,0.01,3,-2.219422,-9.550918
7,35,1,15,100,0.01,4,-2.190175,-9.518123
8,20,1,20,100,0.01,3,-2.386283,-9.546420
9,20,1,20,100,0.01,4,-2.206465,-9.371396


In [117]:
gs_model_df[
    gs_model_df['split0_test_neg_mean_squared_error'] == \
    gs_model_df['split0_test_neg_mean_squared_error'].max()]

,param_hidden_nodes,param_hidden_layers,param_epochs,param_batch_size,param_dropout_rate,param_weight_constraint,split0_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error
9,20,1,20,100,0.01,4,-2.206465,-9.371396


# Evaluating model

In [ ]:
model = create_model(hidden_nodes,
                 hidden_layers,
                 dropout_rate,
                 weight_constraint
                )

model.fit(X_train_sel,
          y_train.ravel(),
          epochs=00,
          batch_size=00)

In [ ]:
# get predictions for test set
y_pred = model.predict(X_test_sel).flatten()

# calculate cross entropy loss for test set
test_loss = (
    mean_absolute_error(y_test, y_pred),
    mean_squared_error(y_test, y_pred),
    r2_score(y_test, y_pred)
)
test_loss

In [ ]:
pd.Series(y_pred).describe()

In [ ]:
# refit model on all data
model_full = GradientBoostingRegressor(n_estimators=120,
                                            max_features=0.61,
                                            min_samples_leaf=0.015,
                                            learning_rate=0.035,
                                            criterion='mse',
                                            random_state=0)
model_full.fit(X_sel, y.ravel())

In [ ]:
# get predictions for whole set
y_pred_full = model_full.predict(X_sel).flatten()

# calculate cross entropy loss for test set
test_loss_full = (
    mean_absolute_error(y, y_pred_full),
    mean_squared_error(y, y_pred_full),
    r2_score(y, y_pred_full)
)
test_loss_full

In [ ]:
pd.Series(y_pred_full).describe()

In [ ]:
abs(pd.Series(y.ravel()) - pd.Series(y_pred_full)).describe()

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist(pd.Series(y_pred_full), bins=range(-5,20), alpha=0.75, density=True, label='predicted')
plt.hist(pd.Series(y.ravel()), bins=range(-5,20), alpha=0.75, density=True, label='observed')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(pd.Series(y.ravel()), pd.Series(y_pred_full), alpha=0.1)
plt.plot(np.linspace(-5, 25), np.linspace(-5, 25))
plt.ylabel('Predicted total points')
plt.xlabel('Observed total points')
plt.ylim((0, 7))
plt.show()

## Summary

In [ ]:
pd.DataFrame([
    ('Neural network', test_loss[0], test_loss[1], test_loss[2]),
], columns=['model', 'MAE', 'MSE', 'r2'])

In [ ]:
pd.DataFrame([
    ('Neural network', test_loss_full[0], test_loss_full[1], test_loss_full[2]),
], columns=['model', 'MAE', 'MSE', 'r2'])